In [2]:
# importing necessary package

import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

# importing modules from source

rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils
import src.loader as loader

In [4]:
# Data acquisition
# we use slack data

data_loader = SlackDataLoader("C:/Users/Diriba/Desktop/10AC/Projects/week0_starter_network_analysis/anonymized")
path_channel = "C:/Users/Diriba/Desktop/10AC/Projects/week0_starter_network_analysis/anonymized"
df = data_loader.slack_parser(path_channel)

df.head()

,msg_type,msg_content,sender_name,msg_sent_time,msg_dist_type,time_thread_start,reply_count,reply_users_count,reply_users,tm_thread_end,channel_name,channel
0,message,<https://towardsdatascience.com/how-to-fill-mi...,Anthony Galloway,1661158502.784099,link,0,0,0,0,0,all-week1,anonymized
1,message,"Imputation Using k-NN, Imputation Using Multiv...",Katie Dickerson,1661158692.172369,text,1661158692.172369,2,2,"U03U1GHT39V,U03UJGRN5E0",1661159006.632649,all-week1,anonymized
2,message,<https://towardsdatascience.com/6-different-wa...,Katie Dickerson,1661158702.777369,link,0,0,0,0,0,all-week1,anonymized
3,message,u can use SMOTE or mode to impute categorical ...,Joseph Diaz,1661158767.626859,text,1661158692.172369,0,0,0,0,all-week1,anonymized
4,message,:+1:thanks,Katie Dickerson,1661159006.632649,emoji,1661158692.172369,0,0,0,0,all-week1,anonymized
